# Preprocessing step 1
This notebook works on the raw data, and selects the target patients who have been prescribed with opioids before.

In [84]:
import numpy as np
import pandas as pd
import pickle
import datetime

In [ ]:
data_dir = "../mimic-iv-2.2/hosp/"
pres = pd.read_csv(data_dir + "prescriptions.csv.gz", compression="gzip")

In [8]:
# have a skim on how the data looks like
pres.head()

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,10000032,22595853,11700683,10000032-34,34.0,P76JEQ,2180-05-07 01:00:00,2180-05-07 22:00:00,MAIN,Acetaminophen,...,004490,9.041989e+08,500mg Tablet,NaN,500,mg,1,TAB,NaN,PO/NG
1,10000032,22595853,14779570,10000032-22,22.0,P76JEQ,2180-05-07 00:00:00,2180-05-07 22:00:00,MAIN,Sodium Chloride 0.9% Flush,...,NaN,0.000000e+00,10 mL Syringe,NaN,3,mL,0.3,SYR,3.0,IV
2,10000032,22595853,19796602,10000032-50,50.0,P260SK,2180-05-08 08:00:00,2180-05-07 22:00:00,MAIN,Furosemide,...,008209,5.107901e+10,40mg Tablet,NaN,40,mg,1,TAB,1.0,PO/NG
3,10000032,22595853,20256254,10000032-32,32.0,P76JEQ,2180-05-07 01:00:00,2180-05-07 22:00:00,MAIN,Raltegravir,...,063231,6.022761e+06,400 mg Tablet,NaN,400,mg,1,TAB,2.0,PO
4,10000032,22595853,28781051,10000032-27,27.0,P76JEQ,2180-05-07 00:00:00,2180-05-07 22:00:00,MAIN,Heparin,...,006549,6.332303e+10,5000 Units / mL- 1mL Vial,NaN,5000,UNIT,1,mL,3.0,SC


In [9]:
# the drug names need to be found
narcotic_drugs = ['oxymorphone', 'oxycodone', 'morphine', 'meperidine',
                  'hydromorphone', 'hydrocodone', 'fentanyl', 'codeine', 'buprenorphine', 'levorphanol', 'methadone']
anti_narcotic_drugs = ['methadone', 'naloxone', 'nalorphine']
both_drugs = narcotic_drugs + anti_narcotic_drugs

In [10]:
# Convert drug names to lower case

pres["drug"] = pres["drug"].str.lower()

In [23]:
pres.shape
pres_no_null = pres[pres["drug"].notnull()]

In [24]:
prescriptions_with_narcotic_drugs = pres_no_null[pres_no_null["drug"].str.contains("|".join(narcotic_drugs))]
prescriptions_with_anti_narcotic_drugs = pres_no_null[pres_no_null["drug"].str.contains("|".join(anti_narcotic_drugs))]
prescriptions_with_both_drugs = pres_no_null[pres_no_null["drug"].str.contains("|".join(both_drugs))]

In [29]:
# Drop buprenorphine-naloxone (8mg-2mg) frm ant-narcotics

prescriptions_with_anti_narcotic_drugs = prescriptions_with_anti_narcotic_drugs[
    prescriptions_with_anti_narcotic_drugs["drug"] != ('buprenorphine-naloxone (8mg-2mg)')]

In [31]:
# print the number of patients with prescriptions
print("total patient number:", len(set(pres['subject_id'])))
print("number of patients w/ prescriptions:", len(set(prescriptions_with_both_drugs['subject_id'])))

total patient number: 158421
number of patients w/ prescriptions: 115488


In [32]:
# identify the subjects prescribed with drugs
subjects_with_narcotic_drugs = set(prescriptions_with_narcotic_drugs["subject_id"])
subjects_with_anti_narcotic_drugs = set(prescriptions_with_anti_narcotic_drugs["subject_id"])
subjects_with_both_drugs = set(prescriptions_with_both_drugs["subject_id"])

In [158]:
print("subjects_with_narcotic_drugs: %i" %(len(subjects_with_narcotic_drugs)))
print("subjects_with_anti_narcotic_drugs: %i" %(len(subjects_with_anti_narcotic_drugs)))
print("subjects_with_both_drugs: %i" %(len(subjects_with_both_drugs)))
print(len(subjects_with_narcotic_drugs.union()))
print(prescriptions_with_both_drugs["drug"].str.contains("methadone").sum())
for drug in both_drugs:
    print(drug, prescriptions_with_both_drugs["drug"].str.contains(drug).sum())

subjects_with_narcotic_drugs: 115432
subjects_with_anti_narcotic_drugs: 10289
subjects_with_both_drugs: 115488
115488
16541
oxymorphone 16
oxycodone 300722
morphine 176151
meperidine 7502
hydromorphone 283860
hydrocodone 10661
fentanyl 60838
codeine 13115
buprenorphine 2385
levorphanol 1
methadone 16541
methadone 16541
naloxone 16146
nalorphine 0


In [36]:
# save the subjects' id
with open('../mimic_data_after_preprocess/subjects.pickle', 'wb') as pickler:
    pickle.dump(subjects_with_both_drugs, pickler, protocol=pickle.HIGHEST_PROTOCOL)

## Preprocessing step 2
With the subjects' id, we can now get the features for each subject from various mimic datasets
columns = [subject_id, age group flag, gender, # of hosp,
            flag of anti-narcotic, narcotic,
            # of each drug,
            total # of anti-narcotic, narcotic,
            diagnose adverse event flag]
Not use levorphanol and nalorphine
In total 3+4+2+11+2+1=23

Age group 1: <13
2: 13-19
3: 20-40
4: 41-50
5: 51-65
6: 66-75
7: 76-85
8: >85

In [77]:
# read patients' information, all admissions information and all icd diagnoses
patients = pd.read_csv(data_dir + "patients.csv.gz", compression="gzip")
admissions = pd.read_csv(data_dir + "admissions.csv.gz", compression="gzip")
diagnoses_icd = pd.read_csv(data_dir + "diagnoses_icd.csv.gz", compression="gzip")

In [71]:
# 1. process patients' information, including gender, age group flag

def turn_age_to_group(age):
    if age<13:
        ret = 'age_1'
    elif age<=19:
        ret = 'age_2'
    elif age<=40:
        ret = 'age_3'
    elif age<=50:
        ret = 'age_4'
    elif age<=65:
        ret = 'age_5'
    elif age<=75:
        ret = 'age_6'
    elif age<=85:
        ret = 'age_7'
    else:
        ret = 'age_8'
    return ret

def process_patients(subject, patients, admissions):
    # _, gender, anchor_age, anchor_year, anchor_year_group, dod = patients[patients['subject_id']==subject]
    pat = patients[patients['subject_id']==subject]
    gender = pat.at[pat.index[-1], 'gender']
    anchor_age = pat.at[pat.index[-1], 'anchor_age']
    anchor_year = pat.at[pat.index[-1], 'anchor_year']
    anchor_year_group = pat.at[pat.index[-1], 'anchor_year_group']

    ret_gender = 0 if gender=="F" else 1
    real_year = int(anchor_year_group[:4])

    sub_adms = admissions[admissions['subject_id']==subject]
    first_adm = sub_adms.iloc[0]

    first_adm_year = int(first_adm['admittime'][:4])
    real_age = anchor_age+first_adm_year-anchor_year
    ret_age_group = turn_age_to_group(real_age)

    return ret_gender, ret_age_group


In [111]:
# 2. get 'n_hosp', 'anti_narcotic', 'narcotic', 'adverse_flag'

def process_overall_hosp(subject, admissions, subjects_with_narcotic, subjects_with_anti_narcotic,
                         subjects_with_effects):
    n_hosp = len(admissions[admissions['subject_id']==subject])
    anti_narcotic = 1 if subject in subjects_with_anti_narcotic else 0
    narcotic = 1 if subject in subjects_with_narcotic else 0
    adverse_flag = 1 if subject in subjects_with_effects else 0

    return n_hosp, anti_narcotic, narcotic, adverse_flag

In [73]:
# still in step 2, find all subjects with adverse effects
# defind all related icd codes here

opioids_effects_list = ["30400", "30401", "30402", "30403", "30470", "30471", "30472",
                               "30473", "30550", "30551", "30552", "30553", "96500", "96501", "96502", "96509"]
psychological_effects_list = ["30410", "30411", "30412", "30413", "30540", "30541", "30542",
                              "30543"]
cocaine_dependence_list = ["30420", "30421", "30422", "30423"]
cannabis_dependence_list = ["30430", "30431", "30432", "30433"]
amphetamine_psychostimulant_list = ["30440", "30441", "30442", "30443"]
hallucinogen_list = ["30450", "30451", "30452", "30453"]
poisoning_list = ["96502", "96509", "9701", "E8500", "E8501", "E8502"]
total_effects_list = opioids_effects_list + psychological_effects_list + cocaine_dependence_list + \
                     cannabis_dependence_list + amphetamine_psychostimulant_list + hallucinogen_list + poisoning_list

In [157]:
diagnoses_with_effects_df = diagnoses_icd[diagnoses_icd["icd_code"].isin(total_effects_list)]
subjects_with_effects = set(diagnoses_icd[diagnoses_icd["icd_code"].isin(total_effects_list)]["subject_id"])
print("Total subjects with effects: %i" %len(subjects_with_effects))
print("Total diagnoses with effects: %i" %len(diagnoses_with_effects_df))
print(len(subjects_with_effects.intersection(subjects_with_both_drugs)))

Total subjects with effects: 3762
Total diagnoses with effects: 7762
2684


In [162]:
# 3. get the amount fo different drugs patients have been subscribed

def calculate_days(row):
    start_date = row["starttime"]
    start_date = start_date.split()[0].split("-", 4)
    end_date = row["stoptime"]
    end_date = end_date.split()[0].split("-", 4)
    no_of_days = datetime.date(int(end_date[0]), int(end_date[1]), int(end_date[2])) - datetime.date(int(start_date[0]), int(start_date[1]), int(start_date[2]))
    return no_of_days.days

def process_all_drugs(subject, prescriptions):
    drug_names = ['oxymorphone', 'oxycodone', 'morphine', 'meperidine',
                  'hydromorphone', 'hydrocodone', 'fentanyl', 'codeine', 'buprenorphine',
                  'methadone', 'naloxone']
    prescriptions = prescriptions[prescriptions['subject_id']==subject]
    ret_dict = {}
    for drug in drug_names:
        patient_pres = prescriptions[prescriptions["drug"].str.contains(drug)]
        drug_days = 0

        for index, row in patient_pres.iterrows():
            if type(row['starttime'])==str and type(row['stoptime'])==str:
                drug_days += max(calculate_days(row), 1)
            else:
                drug_days += 1
        ret_dict[drug] = drug_days
    ret_dict['n_anti_narcotic'] = sum([ret_dict[d] for d in drug_names[-2:]])
    ret_dict['n_narcotic'] = sum([ret_dict[d] for d in drug_names[:-1]])
    return ret_dict


In [163]:
# initialize the dataframe used to store all features
all_data = pd.DataFrame(columns=['age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7', 'age_8',
                                 'gender', 'n_hosp', 'anti_narcotic', 'narcotic',
                                 'n_anti_narcotic', 'n_narcotic',
                                 'oxymorphone', 'oxycodone', 'morphine', 'meperidine',
                                 'hydromorphone', 'hydrocodone', 'fentanyl', 'codeine',
                                 'buprenorphine', 'methadone', 'naloxone',
                                 'adverse_flag'], index=sorted(list(subjects_with_both_drugs)))
all_data.index.name = 'subject_id'

In [164]:
print(all_data.shape)
all_data

(115488, 26)


,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,gender,n_hosp,...,morphine,meperidine,hydromorphone,hydrocodone,fentanyl,codeine,buprenorphine,methadone,naloxone,adverse_flag
subject_id,,,,,,,,,,,,,,,,,,,,,
10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19999464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19999565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19999784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
# Process with each patient one by one
# remind ['age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7', 'age_8',
#                                  'gender', 'n_hosp', 'anti_narcotic', 'narcotic',
#                                  'n_anti_narcotic', 'n_narcotic',
#                                  'oxymorphone', 'oxycodone', 'morphine', 'meperidine',
#                                  'hydromorphone', 'hydrocodone', 'fentanyl', 'codeine',
#                                  'buprenorphine', 'methadone', 'methadone', 'naloxone',
#                                  'adverse_flag']
step = 0
print("Start!")
for subject in sorted(list(subjects_with_both_drugs)):
    gender, age_group = process_patients(subject, patients, admissions)
    n_hosp, anti_narcotic, narcotic, adverse_flag = process_overall_hosp(subject, admissions, subjects_with_narcotic_drugs,
                                                           subjects_with_anti_narcotic_drugs, subjects_with_effects)
    ret_dict = process_all_drugs(subject, prescriptions_with_both_drugs)

    all_data.loc[subject, 'gender'] = gender
    for i in range(1, 9):
        all_data.loc[subject, 'age_'+str(i)] = 1 if age_group=='age_'+str(i) else 0
    all_data.loc[subject, 'n_hosp'] = n_hosp
    all_data.loc[subject, 'anti_narcotic'] = anti_narcotic
    all_data.loc[subject, 'narcotic'] = narcotic
    all_data.loc[subject, 'adverse_flag'] = adverse_flag

    for key, value in ret_dict.items():
        all_data.loc[subject, key] = value

    step += 1
    if step%1000 == 0:
        print("Finish ", step)

Start!
Finish  1000
Finish  2000
Finish  3000
Finish  4000
Finish  5000
Finish  6000
Finish  7000
Finish  8000
Finish  9000
Finish  10000
Finish  11000
Finish  12000
Finish  13000
Finish  14000
Finish  15000
Finish  16000
Finish  17000
Finish  18000
Finish  19000
Finish  20000
Finish  21000
Finish  22000
Finish  23000
Finish  24000
Finish  25000
Finish  26000
Finish  27000
Finish  28000
Finish  29000
Finish  30000
Finish  31000
Finish  32000
Finish  33000
Finish  34000
Finish  35000
Finish  36000
Finish  37000
Finish  38000
Finish  39000
Finish  40000
Finish  41000
Finish  42000
Finish  43000
Finish  44000
Finish  45000
Finish  46000
Finish  47000
Finish  48000
Finish  49000
Finish  50000
Finish  51000
Finish  52000
Finish  53000
Finish  54000
Finish  55000
Finish  56000
Finish  57000
Finish  58000
Finish  59000
Finish  60000
Finish  61000
Finish  62000
Finish  63000
Finish  64000
Finish  65000
Finish  66000
Finish  67000
Finish  68000
Finish  69000
Finish  70000
Finish  71000
Finish  

In [166]:
print(all_data.shape)
for i in range(1,9):
    print("Group",i," : ", sum(all_data['age_'+str(i)]))
print("Male:", sum(all_data['gender']), "  Female:", len(all_data)-sum(all_data['gender']) )
print("Anti_narcotic: ", sum(all_data['anti_narcotic']), "  Narcotic: ", sum(all_data['narcotic']))
print("adverse_flag: ", sum(all_data['adverse_flag']))

(115488, 26)
Group 1  :  0
Group 2  :  777
Group 3  :  25435
Group 4  :  14169
Group 5  :  31441
Group 6  :  20259
Group 7  :  15177
Group 8  :  8230
Male: 53385   Female: 62103
Anti_narcotic:  10289   Narcotic:  115432
adverse_flag:  2684


In [168]:
# save the preprocessed data as csv
all_data.to_csv("../mimic_data_after_preprocess/mimic_preprocessed.csv", encoding='utf-8')